<a href="https://colab.research.google.com/github/shraddha-an/llm-prompt-techniques/blob/main/LLM_Prompting_Techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **0. Installations**

In [ ]:
%%bash
pip install -U transformers==4.36.1 accelerate bitsandbytes --quiet

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# **1. Load Model + Tokenizer**

## **i. Model + Tokenizer**

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

model_name = "databricks/dolly-v2-7b"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(model_name,
                                             load_in_8bit = False,
                                             torch_dtype = torch.bfloat16,
                                             device_map = "auto",
                                             trust_remote_code = True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## **iii. Pipeline**

In [ ]:
pipe = pipeline("text-generation",
                model = model,
                tokenizer = tokenizer,
                device_map = "auto",
                torch_dtype = torch.bfloat16)

# **3. Inference**

In [ ]:
def generate_answer(prompt: str, pipe: pipeline = pipe) -> str:
  """
  Prompt the text-generation pipeline to generate a response.

  Args:
    prompt (str): prompt
    pipe (pipeline): Transformers text-generation pipeline consisting of the LLM.

  Return:
    answer (str): LLM response to the prompt
  """
  sequences = pipe(prompt,
                  do_sample = True,
                  return_full_text = False,
                  temperature = 0.2,
                  num_return_sequences = 1)

  answer = sequences[0]['generated_text']

  return answer

# **4. Zero-shot prompting**
Simple prompt asking the LLM to perform a task.


In [ ]:
zs_prompt = """Identify whether the text is Negative or Positive.

Text: I visited a new restaurant last week and it'll be my first and last time eating there. I would not even rate it a 1 star, the food and hygiene was awful, and the staff behaved extremely rude to us."
Sentiment:"""

zs_answer = generate_answer(zs_prompt)
print(zs_answer)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


 Negative


In [ ]:
zs_prompt2 = """Identify whether the text is Negative or Positive.

Text: I visited a new restaurant last week and the food was delicious! The ambience was pleasant and the chef even greeted us halfway through our meal!"

Sentiment:"""

generate_answer(zs_prompt2)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


' Positive'

# **5. Few-shot prompting**

## **Prompt Format for Dolly**
---
Below is an instruction that describes a task. Write a response that appropriately completes the request.

\### Instruction:

Describe the task in detail, as well as the "role" the LLM is supposed to adopt. Be sure to explicitly tell the LLM what *not* to do in the instructions.


*I added this bit ⬇️ to create a few-shot prompt*

\### EXAMPLES
1. Example 1.

  Output = <response_to_the_task>

2. Example 2.

  Output = <response_to_the_task>

3. Test sentence.

\### Response:

Output =





\### End


In [ ]:
# some test sentences
t1 = "That was an awesome class and an even cooler teacher. I liked how she was patient and was good at explaining things. Definitely will take the class again."
t2 = "Hmm this was terrible. What a waste of my time!"

In [ ]:
INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
INSTRUCTION_KEY = "### Instruction:"
INPUT_KEY = "Input:"
RESPONSE_KEY = f"### Response:\nSentiment = "
END_KEY = "### End"

## **a. Positive**

In [ ]:
test = t1

example_string = f"""
You will be given a few example sentences with their correct sentiment. Learn to identify the sentiment of the text and predict whether it is Negative, Positive or Neutral.
Only respond with the sentiment label.

### EXAMPLES
1. This is awesome!
Sentiment = Positive

2. This is bad!
Sentiment = Negative

3. Well not much to speak about, the museum was average.
Sentiment = Neutral

4. {test}
"""

In [ ]:
PROMPT_FOR_GENERATION_FORMAT = """{intro}

{instruction_key}
{instruction}
{response_key}

{end_key}
""".format(intro = INTRO_BLURB,
           instruction_key = INSTRUCTION_KEY,
           instruction = example_string,
           response_key = RESPONSE_KEY,
           end_key = END_KEY)

print(PROMPT_FOR_GENERATION_FORMAT)


Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:

You will be given a few example sentences with their correct sentiment. Learn to identify the sentiment of the text and predict whether it is Negative, Positive or Neutral. 
Only respond with the sentiment label.

### EXAMPLES
1. This is awesome!
Sentiment = Positive

2. This is bad! 
Sentiment = Negative

3. Well not much to speak about, the museum was average.
Sentiment = Neutral

4. That was an awesome class and an even cooler teacher. I liked how she was patient and was good at explaining things. Definitely will take the class again.

### Response:
Sentiment = 

### End



In [ ]:
print(generate_answer(PROMPT_FOR_GENERATION_FORMAT))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Positive


## **b. Negative**

In [ ]:
test = t2

example_2 = f"""
You will be given a few example sentences with their correct sentiment. Learn to identify the sentiment of the text and predict whether it is Negative, Positive or Neutral.
Only respond with the sentiment label.

### EXAMPLES
1. This is awesome!
Sentiment = Positive

2. This is bad!
Sentiment = Negative

3. Well not much to speak about, the museum was average.
Sentiment = Neutral

4. {test}
"""

In [ ]:
PROMPT_FOR_GENERATION_FORMAT = """{intro}

{instruction_key}
{instruction}

{response_key}

{end_key}
""".format(intro = INTRO_BLURB,
           instruction_key = INSTRUCTION_KEY,
           instruction = example_2,
           response_key = RESPONSE_KEY,
           end_key = END_KEY)

print(PROMPT_FOR_GENERATION_FORMAT)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:

You will be given a few example sentences with their correct sentiment. Learn to identify the sentiment of the text and predict whether it is Negative, Positive or Neutral.
Only respond with the sentiment label.

### EXAMPLES
1. This is awesome!
Sentiment = Positive

2. This is bad!
Sentiment = Negative

3. Well not much to speak about, the museum was average.
Sentiment = Neutral

4. Hmm this was terrible. What a waste of my time!


### Response:
Sentiment = 

### End



In [ ]:
print(generate_answer(PROMPT_FOR_GENERATION_FORMAT))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Negative
